In [56]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

# sites is a list of locations (e.g., "Calhoun"),
# all_pits is a dict with sites as keys, and lists of pits 
# at each site as the values
all_pits = {}
sites = os.listdir('../../processed_data')

# Loop through all sites to construct dict of sites/pits
for site in sites:
    cur_pits = []
    for pro_file in os.listdir(os.path.join('../../processed_data/', site)):
        cur_pits.append(pro_file.split('_')[0])
        
    all_pits[site] = cur_pits

# Data will be a dict with pits as keys, np arrays as values
data = {}
# This is a list of all the features for R1C1. We'll want to make this an exhaustive list
# of all potential features across all pits
features = ['CO2', 'precip', 'SoilMoisture', 'BulkEC', 'Temp', 'O2']
m = len(features)

# Load in and merge all files
# NOTE: Only doing Calhoun R1C1 for now, but you get the idea
i = 0
for site in all_pits.keys():
    for pit in all_pits[site]:
        
        if pit == 'R1C1':
            infile = os.path.join('../../processed_data', site, '%s_processed.csv' % pit)
            df = pd.read_csv(infile, parse_dates=[0], index_col=0, na_values=[-99999])
            depths = [col.split('_')[1].split('cm')[0] for col in df.columns]
            depths = [d for d in depths if d != 'precip.mm']
            unique_depths = list(set(depths))
            
            # t is the number of time steps
            t = df.shape[0]
            
            # data shape is the # of observations (# time points * depths), # features
            cur_data = np.empty((t*len(unique_depths), m), dtype=float)
            
            for i, depth in enumerate(unique_depths):
                depth_cols = [col for col in df.columns if '_%scm' % depth in col]
                for j, feature in enumerate(features):
                    # Look for columns with this feature and depth in the name
                    if feature == 'precip':
                        col = 'PRISM_precip.mm'
                    else:
                        matches = [col for col in depth_cols if feature in col]
                        
                        # Correct for fact that "O2" search returns "CO2" as well
                        if feature == 'O2':
                            matches = [col for col in matches if 'CO2' not in col]
                        
                        if len(matches) > 1:
                            raise ValueError("""More than one possible match found for %s %s %scm %s""" %(site, pit, depth, feature))
                        elif len(matches) == 1:
                            col = matches[0]
                        else:
                            continue

                    # Finally, now that we have the column name
                    cur_data[i*t:(i+1)*t, j] = df[col].values
                    
            data[pit] = cur_data.copy()
            

In the above, we'll need to make sure all the data is in the same units, across all sites

Basically, the goal is to build up a numpy array where each row is an observation (an individual site/depth/time; e.g., Calhoun R1C1 on January 1st at 12:15 pm) and each column is a parameter (e.g., CO2, Soil moisture, O2, etc). We'll have lots of NaN values, and that's fine for now.

What I would recommend doing is building up one site/pit at a time, then combining arrays later on. For example, the R1C1 array should look like:

|     |CO2|precip|SoilMoisture|BulkEC|Temp|O2|
|---|---|---|---|---|---|---|
|12/12/20 12:15 pm @ 25 cm | 5000 ppm | 0 | 0.24 | NaN | 4.37 | 19.1 |
|12/12/20 12:30 pm @ 25 cm | 5169 ppm | 0 | 0.26 | NaN | 4.45 | 19.2 |
|12/12/20 12:45 pm @ 25 cm | 5120 ppm | 0 | 0.29 | NaN | 4.42 | 19.1 |
|12/12/20 01:00 pm @ 25 cm | 5148 ppm | 0 | 0.26 | NaN | 4.49 | 19.2 |
| ... | ... | ... | ... | ... | ... | ... |
|07/04/18 09:15 am @ 150 cm | 6952 ppm | 0.01 | 0.39 | NaN | 4.3 | 19.1 |
| ... | ... | ... | ... | ... | ... | ... |
| etc | etc | etc | etc | etc | etc | etc |

Except without the columns or index labels. You could also set it up as a pandas dataframe (ie, with column and index labels) then extract the values later on. Whichever is easier.